# Benchmarking Our Algorithm against others

In [1]:
import sys
sys.path.append('../../')
from time import time
import signal
from os import listdir
import numpy as np
import pickle as pkl


%load_ext autoreload
%autoreload 2

In [3]:
def get_env_name(filename):
    fs = filename.split('.')
    return fs[0]


def run_experiments(env_name, 
                    env_info, 
                    N_range, 
                    algs, 
                    TIMEOUT = 100,
                    results = [], 
                    executed = None):
    
    '''
        Checkpointed results and timeouted can be passed in so we can resume operation.
    '''

    if executed is None:
        executed = {alg: {N :False for N in N_range} for alg in algs}

    for N in N_range: 
        # Construct the environment
        constructor = env_info['constructor']
        specs = env_info['specs']

        print('Building the environment...')
        env = constructor(**specs,N_agent = N)

        # Run the experiment on each algorithm
        for name,alg in algs.items():

            if executed[name][N]:
                continue
            
            print("Env:", env_name, "Alg:", name, "N agent:",N)

            result = {}
            
            def timeout_handler(signum,frame):
                print("Algorithm timeout")
                # Mark the rest of the executed table as True, so we don't ever visit this algorithm again
                for n in executed[name]:
                    if n>=N:
                        executed[name][n] = True
                # Save checkpoint data
                with open("./data/{}.pkl".format(env_name),'wb') as fp:
                    pkl.dump({'results':results, 'executed':executed},fp)
                raise TimeoutError("Timeout")
            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                t0 = time()
                plan = alg(env)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["env"]=env

            results.append(result)

            # Mark the executed status as True
            executed[name][N] = True

            if t>TIMEOUT:
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                result['plan'] = plan
                print("On time for algorithm", name, "N agent = ", N)

            # Save checkpoint data
            with open("./data/{}.pkl".format(env_name),'wb') as fp:
                pkl.dump({'results':results, 'executed':executed},fp)
            
def get_checkpoint_state(env_name,algs,N_range,first_run = False):
    if first_run:
        results = []
        executed = {alg: {N :False for N in N_range} for alg in algs}
        # Save checkpoint state
        with open("./data/{}.pkl".format(env_name),'wb') as fp:
            pkl.dump({'results':results, 'executed':executed},fp)

    # Load checkpoint state
    with open("./data/{}.pkl".format(env_name),'rb') as fp:
        ckpt = pkl.load(fp)

    results = ckpt['results']
    executed = ckpt['executed']

    return results,executed


In [1]:
from panav.sequential import sequential_HybridSIPP, sequential_planning
from panav.SAMP.solvers import Tube_Planning
from benchmarks.ST_RRTStar_wrapper import sequential_ST_RRTStar 
from panav.PIBT import PIBT_plan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP


bloating_r = 0.5
vmax = 1.0

TIMEOUT = 200

# Specify the Algorithms
algs = {  
        'HybridSIPP':lambda HG: sequential_HybridSIPP(HG,Kmax = 3),
        "Seq_Tube": lambda HG: sequential_planning(Tube_Planning,HG.env,vmax,bloating_r,TIMEOUT=TIMEOUT), 
        "ST-RRT*": lambda HG: sequential_ST_RRTStar(HG.env,vmax,bloating_r),
        "HG-PIBT": lambda HG: PIBT_plan(HG,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "S2M2": lambda HG: PBS(HG.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "HG-MILP": lambda HG: TAHP(HG,vmax,bloating_r,TIMEOUT)
        }
# Load environment presets
env_names = [get_env_name(fn) for fn in listdir('./envs')]

env_info = {}

for name in env_names:
    with open(f'./envs/{name}.pkl','rb') as fp:
        env_info[name] = pkl.load(fp)

ModuleNotFoundError: No module named 'panav'

In [4]:
env_name = 'Room'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed
N_range = np.arange(10,101,10)

In [5]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: Room Alg: HybridSIPP N agent: 10
0
1
2
3
4
5
6
7
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-15


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


8
9
On time for algorithm HybridSIPP N agent =  10
Env: Room Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube N agent =  10
Env: Room Alg: ST-RRT* N agent: 10
Planning for agent 0/10
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


Planning for agent 1/10Info:    SpaceTimeRRT: Created 101 states (72 start + 29 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 19/20Info:    SpaceTimeRRT: Created 251 states (73 start + 178 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


On time for algorithmInfo:    SpaceTimeRRT: Created 222 states (43 start + 179 goal)
 ST-RRT* N agent =  20
Env: Room Alg: HG-PIBT N agent: 20
################# Time step 0 ################
Remaining agents 20
################# Time step 1 ################
Remaining agents 20
################# Time step 2 ################
Remaining agents 20
################# Time step 3 ################
Remaining agents 20
################# Time step 4 ################
Remaining agents 20
################# Time step 5 ################
Remaining agents 20
################# Time step 6 ################
Remaining agents 20
################# Time step 7 ################
Remaining agents 20
################# Time step 8 ################
Remaining agents 20
################# Time step 9 ################
Remaining agents 20
################# Time step 10 ################
Remaining agents 20
################# Time step 11 ################
Remaining agents 20
################# Time step 12 ################
Rem

AttributeError: 'numpy.ndarray' object has no attribute 'A'

In [6]:
env_name = 'MultiTunnel'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(10,101,10)

In [4]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)

# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

NameError: name 'env_name' is not defined

In [10]:
env_name = 'WareHouse'
SEED = 13981
np.random.seed(SEED)  # Fix the random seed

N_range = np.arange(10,101,10)

In [11]:
results,executed = get_checkpoint_state(env_name,algs,N_range,first_run = True)
# results,executed = get_checkpoint_state(env_name,algs,N_range)
run_experiments(env_name,
                env_info[env_name],
                N_range,
                algs,
                TIMEOUT,
                results,
                executed)

Building the environment...
Env: WareHouse Alg: HybridSIPP N agent: 10
0
1
2
3
4
5
6
7
8
9
On time for algorithm HybridSIPP N agent =  10
Env: WareHouse Alg: Seq_Tube N agent: 10
Planning for agent 0/10
Planning for agent 1/10
Planning for agent 2/10
Planning for agent 3/10
Planning for agent 4/10
Planning for agent 5/10
Planning for agent 6/10
Planning for agent 7/10
Planning for agent 8/10
Planning for agent 9/10
On time for algorithm Seq_Tube N agent =  10
Env: WareHouse Alg: ST-RRT* N agent: 10
Planning for agent 0/10
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 1/10
Info:    SpaceTimeRRT: Created 48 states (21 start + 27 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 2/10Info:    SpaceTimeRRT: Created 24 states (11 start + 13 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00
Planning for agent 3/10
Info:    SpaceTimeRRT: Created 50 states (22 start + 28 goal)
Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp
         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


Planning for agent 4/10Info:    SpaceTimeRRT: Created 15 states (7 start + 8 goal)

Debug:   SpaceTimeRRT: Planner range detected to be inf
Info:    SpaceTimeRRT: Starting planning with 1 states already in datastructure
Info:    SpaceTimeRRT: Starting planning with time bound factor 2.00


...

         at line 63 in /Users/tianpengzhang/Desktop/ompl/src/ompl/base/src/SpaceInformation.cpp


################# Time step 8 ################
Remaining agents 10
################# Time step 9 ################
Remaining agents 10
################# Time step 10 ################
Remaining agents 10
################# Time step 11 ################
Remaining agents 10
################# Time step 12 ################
Remaining agents 10
################# Time step 13 ################
Remaining agents 10
################# Time step 14 ################
Remaining agents 10
################# Time step 15 ################
Remaining agents 10
################# Time step 16 ################
Remaining agents 10
################# Time step 17 ################
Remaining agents 10
################# Time step 18 ################
Remaining agents 10
################# Time step 19 ################
Remaining agents 10
################# Time step 20 ################
Remaining agents 10
################# Time step 21 ################
Remaining agents 10
################# Time step 22 ################
Re

AttributeError: 'numpy.ndarray' object has no attribute 'A'

Algorithm timeout


: 